* C-shunted profiling
* Inbuilt-Symmetry model
* Uniform Initialization, Optimal Enclosure : 2dim=5x5 ; 4dim=3x3x3x3
* Parameter subspace : I : fixed : 800GHz
    - A : J1==J3 / J2 ; C1==C3 = 600GHz, C2 = 10GHz
    - B : C1==C3 / C2 ; J1==J3 = 150GHz, J2 = 100GHz
    - C : J1==J3 / C1==C3 ; J2 = 100GHz, C2 = 10GHz
    - D : J1==J3 / J2 / C1==C3 / C2
* Algorithms :
    - SGD : learning rate={.01,.0001} : SGD2,SGD4
    - Adam : learning rate={.01,.0001} : Adam2,Adam4
    - Nelder Mead : !!! Unconstrained Symmetry on Nelder Mead
    - Conjugate Gradient !!! Unconstrained Symmetry on Nelder Mead
    - L-BFGS :
        - scipy : *assummed Wolfe
        - torch : Wolfe
        - torch : lr = .0001
* Benchmarkers Optimization : multi-initialization
    - Loss : min,max,avg : Box Plots
    - Termination : count,steps:min,max,avg

In [1]:
from multiprocess import Pool
#from multiprocessing import Pool
from DiSuQ.Torch.optimization import uniformParameters,initializationParallelism,truncNormalParameters,initializationSequential
from DiSuQ.Torch import models
from DiSuQ.Torch.components import J0,L0,C0
from DiSuQ.Torch.optimization import OrderingOptimization,PolynomialOptimization
from DiSuQ.Torch.optimization import lossTransition,Adam,SGD
from scipy.optimize import LinearConstraint
from torch import tensor, float32 as float
from numpy import arange,linspace,array
from DiSuQ.utils import plotCompare,plotOptimization,plotBox
import pickle
from torch import set_num_threads
set_num_threads(32)

In [2]:
with open('../C-shuntedProfiling/target_fluxqubit.p', 'rb') as content: target_info = pickle.load(content)
target_spectrum = target_info['spectrum']
E10 = target_spectrum[:,1] - target_spectrum[:,0]
E20 = target_spectrum[:,2] - target_spectrum[:,0]
target = {'E10':E10[[0,20,-1]],'E20':E20[[0,20,-1]]}

In [3]:
basis = {'O':[5],'J':[5,5],'I':[]}; rep = 'K'
C12 = .5 ; C22 = 20. ; C11 = C22
JJ1 = 150; JJ3 = 150 ; JJ2 = 50
ind = 800
circuit = models.shuntedQubit(basis,josephson=[JJ1,JJ2,JJ3],cap=[C11,C12,C22],ind=ind,sparse=False,symmetry=True)
static = circuit.circuitState()

In [4]:
Algo = ["SGD2","SGD4","Adam2","Adam4","scipyLBFGS","wolfeLBFGS","lrBFGS"]

In [5]:
lossFunction = lossTransition(tensor(target['E10'],dtype=float),tensor(target['E20'],dtype=float))

In [6]:
optim = OrderingOptimization(circuit,representation=rep)
iterations = 10

In [7]:
flux_range = linspace(0,1,3,endpoint=True)
flux_profile = [{'I':flux} for flux in flux_range]

In [8]:
optimizer = OrderingOptimization(circuit,representation=rep)

In [9]:
symmetry = array([[1,0,0,0,-1,0,0],[0,1,0,0,0,-1,0]])
symmetry = LinearConstraint(symmetry,[0,0],[0,0])

In [10]:
def printLog(dLog):
    print(dLog['loss'].iloc[-1])

In [11]:
def benchmarking(optimizer,initials):
    SGD2,SGD4,Adam2,Adam4,scipyLBFGS,wolfeLBFGS,lrBFGS = [],[],[],[],[],[],[]
    for index,parameter in enumerate(initials):
        print(parameter)
        # Adam
        optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()
        Adam2.append(optimizer.optimization(lossFunction,flux_profile,iterations=iterations,lr=.01))
        optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()
        Adam4.append(optimizer.optimization(lossFunction,flux_profile,iterations=iterations,lr=.0001))
        # SGD
        optimizer.algo = SGD
        optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()
        SGD2.append(optimizer.optimization(lossFunction,flux_profile,iterations=iterations,lr=.01))
        optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()
        SGD4.append(optimizer.optimization(lossFunction,flux_profile,iterations=iterations,lr=.0001))
        # Nelder Mead
        #optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()
        #NelMea.append(optimizer.minimization(lossFunction,flux_profile,method='Nelder-Mead',constraints=symmetry))
        # Conjugate Gradient
        #optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()
        #CG.append(optimizer.minimization(lossFunction,flux_profile,method='CG',constraints=symmetry))
        # LBFGS
        optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()
        scipyLBFGS.append(optimizer.minimization(lossFunction,flux_profile,method='BFGS',constraints=symmetry))
        
        optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()        
        wolfeLBFGS.append(optimizer.optimizationLBFGS(lossFunction,flux_profile,iterations=iterations))
        optimizer.circuit.initialization(parameter); optimizer.parameters,_ = optimizer.circuitParameters()
        lrBFGS.append(optimizer.optimizationLBFGS(lossFunction,flux_profile,iterations=iterations,lr=.0001))
        
    return SGD2,SGD4,Adam2,Adam4,scipyLBFGS,wolfeLBFGS,lrBFGS

In [12]:
def optimizationAnalysis(init,subspace,Search,success=1.):
    Loss,Success,Paths = [],[],[]
    for index,(init,(dLogs,dParams,dCircuit)) in enumerate(zip(init,Search)):
        Paths.append(dCircuit[subspace].to_numpy())
        loss = dLogs['loss'].to_numpy()
        Loss.append(loss[-1])
        if loss[-1] < success:
            Success.append(len(loss))
    return Paths,Loss,Success

In [13]:
def analysisPlotting(Optimization):
    paths = dict()
    losse = dict()
    for algo,(Paths,Loss,Success) in zip(Algo,Optimization):
        for idx,traject in enumerate(Paths):
            paths[algo+str(idx)] = traject
        losse[algo] = Loss
    return paths,losse

### simulation

#### subspace A

In [14]:
subspace = ['JJ1','JJ2'] ; N = 15
initials = uniformParameters(circuit,subspace,N)
initials = truncNormalParameters(circuit,subspace,N,var=5)
len(initials)

15

In [ ]:
Optimization = benchmarking(optimizer,initials)
SGD2,SGD4,Adam2,Adam4,scipyLBFGS,wolfeLBFGS,lrBFGS = Optimization

{'JJ1': 152.71796518082178, 'C1': 20.0, 'JJ2': 54.37019900166661, 'C2': 0.4999997913837433, 'JJ3': 150.00001525878906, 'C3': 20.0, 'I': 800.0}


/usr/local/lib/python3.11/site-packages/scipy/optimize/_minimize.py:565: RuntimeWarning:

Method BFGS cannot handle constraints.



Optimization terminated successfully.
         Current function value: 43.514736
         Iterations: 0
         Function evaluations: 8
         Gradient evaluations: 1
{'JJ1': 146.17828646747182, 'C1': 20.0, 'JJ2': 50.028322881988366, 'C2': 0.4999997913837433, 'JJ3': 150.00001525878906, 'C3': 20.0, 'I': 800.0}


/usr/local/lib/python3.11/site-packages/scipy/optimize/_minimize.py:565: RuntimeWarning:

Method BFGS cannot handle constraints.



Optimization terminated successfully.
         Current function value: 22.667871
         Iterations: 0
         Function evaluations: 8
         Gradient evaluations: 1
{'JJ1': 158.67595513977514, 'C1': 20.0, 'JJ2': 51.67137533259122, 'C2': 0.4999997913837433, 'JJ3': 150.00001525878906, 'C3': 20.0, 'I': 800.0}


...

Optimization terminated successfully.
         Current function value: 44.264271
         Iterations: 0
         Function evaluations: 8
         Gradient evaluations: 1
{'JJ1': 150.47184005549164, 'C1': 20.0, 'JJ2': 52.861773649561336, 'C2': 0.4999997913837433, 'JJ3': 150.00001525878906, 'C3': 20.0, 'I': 800.0}


/usr/local/lib/python3.11/site-packages/scipy/optimize/_minimize.py:565: RuntimeWarning:

Method BFGS cannot handle constraints.



Optimization terminated successfully.
         Current function value: 35.530170
         Iterations: 0
         Function evaluations: 8
         Gradient evaluations: 1
{'JJ1': 141.05806040226713, 'C1': 20.0, 'JJ2': 47.20495234945068, 'C2': 0.4999997913837433, 'JJ3': 150.00001525878906, 'C3': 20.0, 'I': 800.0}


/usr/local/lib/python3.11/site-packages/scipy/optimize/_minimize.py:565: RuntimeWarning:

Method BFGS cannot handle constraints.



Optimization terminated successfully.
         Current function value: 12.118976
         Iterations: 0
         Function evaluations: 8
         Gradient evaluations: 1


In [ ]:
Result = [optimizationAnalysis(initials,subspace,algo) for algo in Optimization]
paths,losse = analysisPlotting(Result)

In [ ]:
plotBox(losse,'Loss Distribution-JJ1-JJ2',export='pdf',size=(600,800))

In [ ]:
JJ1 = linspace(0,J0,20,endpoint=True)
JJ2 = linspace(0,J0,20,endpoint=True)
scape = {'JJ1':JJ1,'JJ2':JJ2}

In [ ]:
flux_profile = [{'I':flux} for flux in flux_range]

In [ ]:
Loss = optim.lossScape(lossFunction,flux_profile,scape,static)

In [ ]:
plotOptimization(Loss,JJ1,JJ2,paths,'Optimization-Flux profiling','JJ1(GHz)','JJ2(GHz)',export='pdf',size=(600,800))

#### subspace B

In [ ]:
subspace = ['C1','C2'] ; N = 5
initials = uniformParameters(circuit,subspace,N)

#### subspace C

In [ ]:
subspace = ['J1','C1'] ; N = 5
initials = uniformParameters(circuit,subspace,N)

#### subspace D

In [ ]:
subspace = ['JJ1','JJ2','C1','C2'] ; N = 3
initials = uniformParameters(circuit,subspace,N)